Extracting and Visualizing Stock Data

Objective

In [1]:
!pip install yfinance
!pip install requests
!pip install plotly
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=2dc2f3b3cc4752b43d6bd8f18fd17b4aebbf4ca30f5b584aaa47a87c17cc14e1
  Stored in directory: c:\users\linyi\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [22]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

Using yfinance to Extract Stock Data

In [4]:
Tesla = yf.Ticker("TSLA")

In [5]:
tesla_data = Tesla.history(period="max")
df=tesla_data

In [7]:
df.reset_index(inplace=True)
df

,index,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,0,2010-06-29,3.800000,5.000000,3.508000,4.778000,93831500,0,0.0
1,1,2010-06-30,5.158000,6.084000,4.660000,4.766000,85935500,0,0.0
2,2,2010-07-01,5.000000,5.184000,4.054000,4.392000,41094000,0,0.0
3,3,2010-07-02,4.600000,4.620000,3.742000,3.840000,25699000,0,0.0
4,4,2010-07-06,4.000000,4.000000,3.166000,3.222000,34334500,0,0.0
...,...,...,...,...,...,...,...,...,...
3052,3052,2022-08-12,868.250000,900.479980,855.099976,900.090027,26443300,0,0.0
3053,3053,2022-08-15,905.359985,939.400024,903.690002,927.960022,29786400,0,0.0
3054,3054,2022-08-16,935.000000,944.000000,908.650024,919.690002,29378800,0,0.0
3055,3055,2022-08-17,910.190002,928.969971,900.099976,911.989990,22922000,0,0.0


Extracting Tesla Revenue Data Using Webscraping

In [8]:
url= "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data = requests.get(url).text

In [11]:
soup = BeautifulSoup(html_data,"html.parser")

In [12]:
df=pd.read_html(html_data,header=0)
table=soup.find_all('table')
second_table= table[1]
tesla_revenue= pd.DataFrame(columns=["Date","Revenue"])
for row in second_table.find("tbody").find_all("tr"):
    col= row.find_all('td')
    date= col[0].string
    revenue= col[1].string
    tesla_revenue= tesla_revenue.append({"Date":date, "Revenue":revenue}, ignore_index = True)
tesla_revenue["Revenue"]= tesla_revenue['Revenue'].str.replace('$','').str.replace(',','')
print(tesla_revenue)

          Date Revenue
0   2022-06-30   16934
1   2022-03-31   18756
2   2021-12-31   17719
3   2021-09-30   13757
4   2021-06-30   11958
5   2021-03-31   10389
6   2020-12-31   10744
7   2020-09-30    8771
8   2020-06-30    6036
9   2020-03-31    5985
10  2019-12-31    7384
11  2019-09-30    6303
12  2019-06-30    6350
13  2019-03-31    4541
14  2018-12-31    7226
15  2018-09-30    6824
16  2018-06-30    4002
17  2018-03-31    3409
18  2017-12-31    3288
19  2017-09-30    2985
20  2017-06-30    2790
21  2017-03-31    2696
22  2016-12-31    2285
23  2016-09-30    2298
24  2016-06-30    1270
25  2016-03-31    1147
26  2015-12-31    1214
27  2015-09-30     937
28  2015-06-30     955
29  2015-03-31     940
30  2014-12-31     957
31  2014-09-30     852
32  2014-06-30     769
33  2014-03-31     621
34  2013-12-31     615
35  2013-09-30     431
36  2013-06-30     405
37  2013-03-31     562
38  2012-12-31     306
39  2012-09-30      50
40  2012-06-30      27
41  2012-03-31      30
42  2011-12

C:\Users\linyi\AppData\Local\Temp/ipykernel_18584/3212783899.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tesla_revenue["Revenue"]= tesla_revenue['Revenue'].str.replace('$','').str.replace(',','')


Extracting GameStop Stock Data Using yfinance 

In [13]:
 GME= yf.Ticker("GME")

In [14]:
GME_share_price_data = GME.history(period="max")
df=GME_share_price_data

In [16]:
df.reset_index(inplace=True)
df

,index,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,0,2002-02-13,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,1,2002-02-14,1.712707,1.716073,1.670626,1.683250,11021600,0.0,0.0
2,2,2002-02-15,1.683251,1.687459,1.658002,1.674834,8389600,0.0,0.0
3,3,2002-02-19,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,4,2002-02-20,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0
...,...,...,...,...,...,...,...,...,...
5160,5160,2022-08-12,40.000000,41.430000,39.150002,40.740002,5062800,0.0,0.0
5161,5161,2022-08-15,39.750000,40.389999,38.810001,39.680000,5243100,0.0,0.0
5162,5162,2022-08-16,39.169998,45.529999,38.599998,42.189999,23602800,0.0,0.0
5163,5163,2022-08-17,42.180000,44.360001,40.410000,40.520000,9766400,0.0,0.0


Extracting GameStop Revenue Data Using Webscraping 

In [17]:
url= "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
html_data = requests.get(url).text

In [18]:
soup = BeautifulSoup(html_data,"html.parser")

In [19]:
df=pd.read_html(html_data,header=0)
table=soup.find_all('table')
second_table= table[1]
gme_revenue= pd.DataFrame(columns=["Date","Revenue"])
for row in second_table.find("tbody").find_all("tr"):
    col= row.find_all('td')
    date= col[0].string
    revenue= col[1].string
    gme_revenue= gme_revenue.append({"Date":date, "Revenue":revenue}, ignore_index = True)
gme_revenue["Revenue"]= gme_revenue['Revenue'].str.replace('$','').str.replace(',','')

gme_revenue.dropna(subset=['Revenue'], inplace=True)
for i in gme_revenue : gme_revenue[i] = gme_revenue[i].astype(str)
print(gme_revenue)

          Date Revenue
0   2022-04-30    1378
1   2022-01-31    2254
2   2021-10-31    1297
3   2021-07-31    1183
4   2021-04-30    1277
5   2021-01-31    2122
6   2020-10-31    1005
7   2020-07-31     942
8   2020-04-30    1021
9   2020-01-31    2194
10  2019-10-31    1439
11  2019-07-31    1286
12  2019-04-30    1548
13  2019-01-31    3063
14  2018-10-31    1935
15  2018-07-31    1501
16  2018-04-30    1786
17  2018-01-31    2825
18  2017-10-31    1989
19  2017-07-31    1688
20  2017-04-30    2046
21  2017-01-31    2403
22  2016-10-31    1959
23  2016-07-31    1632
24  2016-04-30    1972
25  2016-01-31    3525
26  2015-10-31    2016
27  2015-07-31    1762
28  2015-04-30    2061
29  2015-01-31    3476
30  2014-10-31    2092
31  2014-07-31    1731
32  2014-04-30    1996
33  2014-01-31    3684
34  2013-10-31    2107
35  2013-07-31    1384
36  2013-04-30    1865
37  2013-01-31    3562
38  2012-10-31    1773
39  2012-07-31    1550
40  2012-04-30    2002
41  2012-01-31    3579
42  2011-10

C:\Users\linyi\AppData\Local\Temp/ipykernel_18584/3174127543.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  gme_revenue["Revenue"]= gme_revenue['Revenue'].str.replace('$','').str.replace(',','')


In [20]:
gme_revenue

,Date,Revenue
0,2022-04-30,1378
1,2022-01-31,2254
2,2021-10-31,1297
3,2021-07-31,1183
4,2021-04-30,1277
5,2021-01-31,2122
6,2020-10-31,1005
7,2020-07-31,942
8,2020-04-30,1021
9,2020-01-31,2194


Tesla Stock and Revenue Dashboard

In [28]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()


make_graph(tesla_data, tesla_revenue,'Tesla')

GameStop Stock and Revenue Dashboard-

make_graph(gme_data, gme_revenue, 'GameStop')